In [1]:
import math
from tensorboardX import SummaryWriter

sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import env_configurations
import games_configurations
env_name = 'SuperMarioBrosRandomStages-v1'#'PongNoFrameskip-v4' #'MountainCar-v0' #'SuperMarioBros-v1'# 'PongNoFrameskip-v4' #SuperMarioBros-v1'
#env_name = 'CartPole-v1'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #

obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
config = games_configurations.mario_random_config_lstm

observation_shape = obs_space.shape
n_actions = action_space.n
shape =(None, ) + observation_shape
print(shape)
print(n_actions)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


(None, 84, 84, 4)
7


In [4]:
from a2c_discrete import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

2019-07-06 21:06:24,445	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-06_21-06-24_2814/logs.
2019-07-06 21:06:24,557	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:47629 to respond...
2019-07-06 21:06:24,700	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:48934 to respond...
2019-07-06 21:06:24,705	INFO services.py:804 -- Starting Redis shard with 6.74 GB max memory.
2019-07-06 21:06:24,765	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-06_21-06-24_2814/logs.
2019-07-06 21:06:24,767	INFO services.py:1427 -- Starting the Plasma object store with 10.11 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:47629',
 'object_store_address': '/tmp/ray/session_2019-07-06_21-06-24_2814/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-06_21-06-24_2814/sockets/raylet',
 'webui_url': None}

In [5]:


agent = A2CAgent(sess,'stages_all', obs_space, True, action_space, config)
#agent.restore('nn/last_skip')
agent.train()


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


2019-07-06 21:06:25,926	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


0
1
2
3
4
5
6
7
0
Instructions for updating:
Use tf.cast instead.
(pid=2931) 
(pid=2931) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=2931) For more information, please see:
(pid=2931)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=2931)   * https://github.com/tensorflow/addons
(pid=2931) If you depend on functionality not listed there, please file an issue.
(pid=2931) 
(pid=2929) 
(pid=2929) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=2929) For more information, please see:
(pid=2929)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=2929)   * https://github.com/tensorflow/addons
(pid=2929) If you depend on functionality not listed there, please file an issue.
(pid=2929) 
(pid=2932) 
(pid=2932) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=2932) For more information, please see:
(pi

(pid=2953) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=2953)   result = entry_point.load(False)
(pid=2927) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=2927)   result = entry_point.load(False)
(pid=2972) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=2972)   result = entry_point.load(False)
(pid=2928) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=2928)   result = entry_point.load

Frames per seconds:  373.13200581308473
Frames per seconds:  378.64918067010746
Frames per seconds:  387.78708728688815
Frames per seconds:  410.1817430297343
Frames per seconds:  404.0885127110326
Frames per seconds:  409.9580862151761
Frames per seconds:  417.02120658795604
Frames per seconds:  398.3259882976591
Frames per seconds:  367.5059539198787
Frames per seconds:  404.0333176218521
Frames per seconds:  403.0091922451996
Frames per seconds:  398.5489641313408
Frames per seconds:  380.9139206473546
Frames per seconds:  391.7408806338206
Frames per seconds:  399.0339028567561
Frames per seconds:  401.34078708433464
Frames per seconds:  403.1047179707282
Frames per seconds:  377.7125139131398
Frames per seconds:  399.9760195089207
Frames per seconds:  404.8085873232024
Frames per seconds:  396.649857419683
Frames per seconds:  402.493765024225
Frames per seconds:  378.0705689274769
Frames per seconds:  388.6241028494507
Frames per seconds:  406.7649163109143
Frames per seconds:  4

...

Frames per seconds:  408.158799785418
Frames per seconds:  409.1719906826208
Frames per seconds:  409.5859575126909
Frames per seconds:  408.31804824198014
Frames per seconds:  377.40227126721925
Frames per seconds:  414.89830455998924
Frames per seconds:  362.85765002894647
Frames per seconds:  277.191413002861
Frames per seconds:  360.6457275310438
Frames per seconds:  377.7816515376204
Frames per seconds:  361.9915804078551
Frames per seconds:  403.16244115731314
Frames per seconds:  338.7678415769112
Frames per seconds:  324.35959042323094
Frames per seconds:  423.6068574927353
Frames per seconds:  329.01278643816585
Frames per seconds:  399.10463246457954
Frames per seconds:  393.2671506529566
Frames per seconds:  352.9631768670035
Frames per seconds:  423.6245937500447
Frames per seconds:  366.9961322414624
Frames per seconds:  386.32999326998134
Frames per seconds:  421.73134811404134
Frames per seconds:  415.69756576492034
Frames per seconds:  419.9430257638719
Frames per secon

Frames per seconds:  283.69116094836926
Frames per seconds:  357.99037227781463
Frames per seconds:  327.63835529328014
Frames per seconds:  409.10051218916874
Frames per seconds:  404.92432227337844
Frames per seconds:  402.06456775506547
Frames per seconds:  396.43448960170724
Frames per seconds:  382.5294398279059
Frames per seconds:  359.12486548915456
Frames per seconds:  376.47395021477587
Frames per seconds:  426.19738646231
Frames per seconds:  426.61990360402694
Frames per seconds:  303.4752595121847
Frames per seconds:  399.24265981668253
Frames per seconds:  408.07646926699977
Frames per seconds:  330.3366402885018
Frames per seconds:  410.0035221432226
Frames per seconds:  386.87127217529013
Frames per seconds:  402.1525292356347
Frames per seconds:  298.8651593634924
Frames per seconds:  410.56319719801627
Frames per seconds:  409.49883358684474
Frames per seconds:  332.18373651619595
Frames per seconds:  363.3394470701336
Frames per seconds:  424.79152625938787
Frames per

Frames per seconds:  380.84344617084935
Frames per seconds:  386.1723613212572
Frames per seconds:  421.6038833007271
Frames per seconds:  407.76700917615676
Frames per seconds:  307.6917612879767
Frames per seconds:  399.40975214460025
Frames per seconds:  411.48074471013285
Frames per seconds:  397.38823692254283
Frames per seconds:  394.68436483430185
Frames per seconds:  332.53081147891163
Frames per seconds:  415.5285101893059
Frames per seconds:  412.3442400735409
Frames per seconds:  413.0593428459525
Frames per seconds:  339.0025432093406
Frames per seconds:  332.68308948522883
Frames per seconds:  325.0538052151515
Frames per seconds:  408.2865495650099
Frames per seconds:  408.13696325542026
Frames per seconds:  388.543804621861
Frames per seconds:  400.37635626775545
Frames per seconds:  407.35033434043055
Frames per seconds:  405.31845046756126
Frames per seconds:  364.31648477635355
Frames per seconds:  395.4289625333499
Frames per seconds:  425.08018894488157
Frames per s

Exception in thread ray_import_thread:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/import_thread.py", line 70, in _run
    msg = import_pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/trrrrr/anaconda3/envs/

KeyboardInterrupt: 

In [6]:
agent.save('nn/last_all')
ray.shutdown()

In [ ]:
gym.envs.registry.all()